In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!unzip /content/drive/MyDrive/Colab_Notebooks/8k_image_captioning.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2 as op
import progressbar
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, add, Conv2D
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import seaborn as sea
from scipy import stats
from keras import layers
from os import listdir
from keras.applications.vgg16 import preprocess_input
from pickle import dump, load

plt.style.use('seaborn')
tf.__version__

ModuleNotFoundError: No module named 'progressbar'

In [ ]:
df = pd.read_csv("captions.txt")

In [ ]:
df.describe()

In [ ]:
n_images = 8091
n_captions = 40455
8091 * 5 == 40455

In [ ]:
base_model = MobileNet(input_shape = (224,224,3), include_top = True)
x = Conv2D(filters = 512, kernel_size = (1,1), activation = 'sigmoid')(base_model.layers[-5].output)
base_model = Model(inputs= base_model.inputs, outputs= x)
base_model.training = False

In [ ]:
base_model.summary()

In [ ]:
def load_preprocess_image(path):
    img = op.imread(path)
    img = op.cvtColor(img, op.COLOR_BGR2RGB)
    img = op.resize(img, (224,224))
    img = img /255.0
    img = img.reshape(1,224,224,3)
    return img


In [ ]:
img = load_preprocess_image('/content/Images/1000268201_693b08cb0e.jpg')
plt.imshow(img[0])

In [ ]:
def extract_features(directory):
    
    # base_model = MobileNet(input_shape = (224,224,3), include_top = True)
    # base_model = Model(inputs= base_model.inputs, outputs= base_model.layers[-5].output)
    # base_model.training = False
    # print(base_model.summary())
    # print(base_model.output.shape)
    features = dict()
    
    for i, name in enumerate(listdir(directory)):
        img = load_preprocess_image(directory + "/" + name)
        fvec = base_model.predict(img, verbose = 0)
        features[name] = fvec

        if((i+1) % 1000 == 0):
            print((i+1),'Done')
    
    return features


In [ ]:
DIR = '/content/Images'

features = extract_features(DIR)

# dump(features, open('features.pkl', 'wb'))

In [ ]:
dump(features, open('features.pkl', 'wb'))
del features

In [ ]:
import string

def remove_punctuation(text_original):
    text_no_punctuation = text_original.translate(str.maketrans('','',string.punctuation))
    return(text_no_punctuation)

def remove_numeric(text,printTF=False):
    text_no_numeric = ""
    for word in text.split():
        isalpha = word.isalpha()
        if printTF:
            print("{:10} : {:}".format(word,isalpha))
        if isalpha:
            text_no_numeric += " " + word
    return(text_no_numeric)

def lower_text(text):
    return str(text).lower()

def add_start_end(caption):
    caps = []
    for cap in caption:
        cap = 'startseq' + cap + ' endseq'
        caps.append(cap)
    return caps


In [ ]:
def text_clean(text_original):
    text = remove_punctuation(text_original)
    text = remove_numeric(text)
    text = lower_text(text)
    return(text)

with progressbar.ProgressBar(max_value=len(df.caption.values)) as bar:
    for i, caption in enumerate(df.caption.values):
        newcaption = text_clean(caption)
        df["caption"].iloc[i] = newcaption
        bar.update(i)

In [ ]:
df['caption'] = add_start_end(df['caption'])

In [ ]:
df['caption'].values

In [ ]:

def get_vocab_len():
    vocabulary = []

    for i in range(len(df)):
        try:
            temp = df.iloc[i,1]
            vocabulary.extend(temp.split())
        except:
            print(i)

    return len(set(vocabulary))


In [ ]:
n_vocab = get_vocab_len()
n_vocab += 1
print(n_vocab)

In [ ]:
tokenizer = Tokenizer(n_vocab)

tokenizer.fit_on_texts(df['caption'].values)

# tokenized_captions = tokenizer.texts_to_sequences(df['caption'].values)


In [ ]:
word2idx = tokenizer.word_index
word2idx['pad'] = 0
idx2word = dict((idx, word) for (word, idx) in word2idx.items())

def decode(seq):
    return " ".join([idx2word.get(idx, "?") for idx in seq])

In [ ]:
def define_model(embedding_dim, seq_len, n_vocab):
    img_input = Input(shape = (512,))
    # fimg = base_model(img)
    # fimg = Dense(512, activation = 'relu')(img_input)
    # fimg = Dropout(0.2)(img_input)

    txt_input = Input(shape = (seq_len, ))
    txt = Embedding(n_vocab, embedding_dim, mask_zero = True)(txt_input)
    txt = LSTM(512, return_sequences = True)(txt)
    txt = Dropout(0.3)(txt)
    txt = LSTM(512)(txt)

    decoder = add([img_input, txt])
    decoder = Dense(1500, activation = 'relu')(decoder)
    decoder = Dropout(0.2)(decoder)

    output = Dense(n_vocab, activation = 'softmax')(decoder)

    model = Model(inputs = [img_input, txt_input], outputs = output)

    model.compile(
        loss = 'categorical_crossentropy', 
        optimizer =  keras.optimizers.Adam())

    return model

In [ ]:
embedding_dim = 128
seq_len = 6

model = define_model(embedding_dim, seq_len, n_vocab)


In [ ]:
model.summary()

In [ ]:

all_features = load(open('/content/features.pkl', 'rb'))
for key in all_features.keys():
    all_features[key] = np.reshape(all_features[key], (512, ))
    

def create_sequences(captions, images, seq_len = 6):
    X1, X2, y = [], [], []

    for cap, img in zip(captions, images):
        cap = cap.split()
        # predicting each next word
        # earlier window was selected and then next word was predicted
        # problem observed was initial words were not predicted during testing, since they weren't predicted during training itself
        for i in range(0, len(cap)-1):
            in_seq = tokenizer.texts_to_sequences([cap[0 : i + 1]])
            in_seq = pad_sequences(in_seq, maxlen = seq_len, padding = 'post')[0]
            out_seq = word2idx[cap[i + 1]]
            
            if(sum(in_seq) == 0):
                break
            
            X1.append(in_seq)
            X2.append(all_features[img])
            y.append(np_utils.to_categorical(out_seq, n_vocab))
    
    return np.array(X1), np.array(X2), np.array(y)

def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features



In [ ]:
# def data_generator(data):
#     i = 0 # Can be inialized to a random index and start the batch from that index cylically for each epoch
#     while 1:
#         x1, x2, y = create_sequences(data.iloc[i: i+ 29, 1], data.iloc[i : i + 29, 0], seq_len)
#         i = (i + 29) % len(df)
#         yield [[x2,x1], y]

#randomized implementation
def data_generator(data):
    i = 0 # Can be inialized to a random index and start the batch from that index cylically for each epoch
    while 1:
        td = data.sample(n = batch_size)
        x1, x2, y = create_sequences(td.iloc[:, 1], td.iloc[:, 0], seq_len)
        # i = (i + 29) % len(df)
        yield [[x2,x1], y]


In [ ]:
loss = []

In [ ]:
epochs = 5 # have to increase. On higher epochs, loss kept decreasing but prediction wasn't good. Performed better for fewer epochs.
batch_size = 29
steps = len(df)/batch_size # NO PROBLEM HERE

for i in range(epochs):
    print("Epoch:", i+1)
    generator = data_generator(df)
    model.fit(generator, epochs = 1, steps_per_epoch = steps)
    loss.append(model.history.history['loss'])


In [ ]:
plt.plot(np.arange(len(loss)) + 1, loss, '--r')

In [ ]:
model.save("model_v4_loss=1.5008.h5")

In [ ]:
from keras.models import load_model

# model2 = load_model('/content/drive/MyDrive/Model/model_v4_loss=1.5008.h5')

def predict_caption(image):
    '''
    image.shape = (1,4462)
    '''
    in_text = 'startseq'

    for iword in range(30):
        sequence = tokenizer.texts_to_sequences([in_text])
        sequence = pad_sequences(sequence, maxlen = seq_len, padding = 'post')[0]
        sequence = np.array(sequence).reshape(1, -1)
    
        yhat = model.predict([image,sequence],verbose=0)
        yhat = np.argmax(yhat)
        
        newword = idx2word[yhat]
        # print(newword)
        in_text += " " + newword
        
        if newword == "endseq":
            break
            
    return(in_text)



In [ ]:
url = 'https://i.insider.com/5afc93865e48ec51008b458a?width=600&format=jpeg&auto=webp'
import requests, io
import numpy as np
from PIL import Image


response = requests.get(url)
bytes_im = io.BytesIO(response.content)
img = op.cvtColor(np.array(Image.open(bytes_im)), op.COLOR_BGR2RGB)

img = op.resize(img, (224,224))
img = op.cvtColor(img, op.COLOR_BGR2RGB)
plt.axis('off')
plt.imshow(img)
img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
img = img/255.0


In [ ]:
# test_path = '/content/45127963-young-black-man-working-in-construction-site.webp'
# img = load_preprocess_image(test_path)

img_vec = base_model.predict(img).reshape(1,-1)


In [ ]:
img_vec.shape

In [ ]:
ans = predict_caption(img_vec)

In [ ]:
ans